In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

In [ ]:
vec_king

In [ ]:
import pandas as pd
messages=pd.read_csv('smsspamcollection/SMSSpamCollection',
                    sep='\t',names=["label","message"])

In [ ]:
messages.shape

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
import re
import nltk
nltk.download('stopwords')

In [ ]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

In [ ]:
corpus

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))


In [ ]:
words

In [ ]:
import gensim

In [ ]:
## Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words)

In [ ]:
## To Get All the Vocabulary
model.wv.index_to_key

In [ ]:
model.corpus_count

In [ ]:
model.epochs

In [ ]:
model.wv.similar_by_word('good')

In [ ]:
model.wv['good'].shape

In [ ]:
words[0]

In [ ]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

In [ ]:
len(X)

In [ ]:
##independent Features
X_new=np.array(X)

In [ ]:
messages.shape

In [ ]:
X[1]

In [ ]:
X_new.shape

In [ ]:
X_new[0].shape

In [ ]:
## Dependent Features
## Output Features
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

In [ ]:
y.shape

In [ ]:
X[0].reshape(1,-1).shape

In [ ]:
## this is the final independent features
df=pd.DataFrame()
for i in range(0,len(X)):
    df=df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)
    

In [ ]:
df.head()

In [ ]:
df['Output']=y

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
## Independent Feature
X=df

In [ ]:
X.isnull().sum()

In [ ]:
y=df['Output']

In [ ]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred=classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))